In [44]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY=os.getenv("GOOGLE_API_KEY")

YOUTUBE_VIDEO="https://www.youtube.com/watch?v=cpLi-82koFM"

In [13]:
from langchain_google_genai import ChatGoogleGenerativeAI

model=ChatGoogleGenerativeAI(model="gemini-pro")

In [15]:
result=model.invoke("What is capital of india")
print(result.content)

New Delhi


In [22]:
from langchain_core.output_parsers import StrOutputParser

parser=StrOutputParser()

chain=model | parser
chain.invoke("What is capital of india")

'New Delhi'

In [23]:
from langchain.prompts import ChatPromptTemplate

template="""
Answer the question based on the context below. If you can't answer the question, reply "I don't know".

Context:{context}
 
Question:{question}
"""
prompt=ChatPromptTemplate.from_template(template)
prompt.format(context="Mary's sister is Susana",question="Who is Mary's sister?")

'Human: \nAnswer the question based on the context below. If you can\'t answer the question, reply "I don\'t know".\n\nContext:Mary\'s sister is Susana\n \nQuestion:Who is Mary\'s sister?\n'

In [24]:
chain= prompt | model | parser
chain.invoke({
            "context":"Mary's sister is Susana",
            "question":"Who is Mary's sister?"
})

'Susana'

### Combining Chains

In [28]:
translation_prompt=ChatPromptTemplate.from_template(
    "Translate {answer} to {language}"
)

In [91]:
from operator import itemgetter

translation_chain=(
    {"answer":chain,"language":itemgetter("language")} | translation_prompt | model | parser 
)
translation_chain.invoke(
    {
        "context":"Mary's sister is Susana. She doesn't have any more siblings.",
        "question":"Who is Mary's sisters? does Mary have one?",
        "language":"Spanish",
    }
)

'La hermana de Mary es Susana. Sí, Mary tiene una hermana.'

In [92]:
with open("transcription.txt") as file:
    transcription=file.read()
transcription[:100]

"\n\nYo, I am so psyched\nfor Invisible Force 2.\n\nPlease. Invisible Force 1 was lame.\n\nI'm all about Ris"

In [97]:
try:
    chain.invoke({
        "context":transcription,
        "question":"What is in the given text?"
    })
except Exception as e:
    print(e)

In [108]:
from vertexai.preview import tokenization


model="gemini-1.5-pro-001"
tokenizer=tokenization.get_tokenizer_for_model(model)
with open("transcription.txt") as file:
    transcription=file.read()
result=tokenizer.count_tokens(transcription)

In [109]:
print(result)

CountTokensResult(total_tokens=28483)
